In [ ]:
import os
import json
import math

import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import rc
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from zoomin_client import client
from dotenv import load_dotenv, find_dotenv

In [ ]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
# load up the entries as environment variables
load_dotenv(dotenv_path)

api_key = os.environ.get("DJANGO_API_Key")

In [ ]:
cwd = os.getcwd()
REPORT_PATH = os.path.join(cwd, "..", "..", "reports", "04_D3.2_appendix")
DATA_PATH = os.path.join(cwd, "..", "..", "data", "output", "api_data")
VAR_DF_PATH = os.path.join(cwd, "..", "..", "data", "input", "raw", "vars_list_details_and_tags.xlsx")
SHP_PATH = os.path.join(cwd, "..", "..", "data", "input", "processed", "shapefiles")

In [ ]:
clm_related_vars = ["annual mean mean temperature",
            "annual mean precipitation",
            "annual total precipitation",
            "annual mean maximum temperature",
            "annual mean minimum temperature",
            "annual mean temperature cooling degree days",
            "annual maximum temperature cooling degree days",
            "annual minimum temperature cooling degree days",
            "annual mean temperature heating degree days",
            "annual maximum temperature heating degree days",
            "annual minimum temperature heating degree days",
                    "probability of a heatwave",
                   "change in heatwave frequency",
                   "probability of a coldwave",
                   "change in coldwave frequency",
                   "probability of a drought",
                   "change in drought frequency"
           ]

eucalc_transpor_related_data = ["tra_vehicle-lifetime_new_freight_HDVL_FCEV",
"tra_vehicle-lifetime_new_freight_HDVL_CEV",
"tra_vehicle-lifetime_new_freight_HDVL_PHEV-diesel",
"tra_vehicle-lifetime_new_freight_HDVL_PHEV-gasoline",
"tra_vehicle-lifetime_new_freight_HDVL_ICE-gasoline",
"tra_vehicle-lifetime_new_freight_HDVL_ICE-diesel",
"tra_vehicle-lifetime_new_freight_HDVL_ICE-gas",
"tra_vehicle-lifetime_new_freight_HDVL_BEV",
"tra_vehicle-lifetime_new_freight_HDVM_FCEV",
"tra_vehicle-lifetime_new_freight_HDVM_CEV",
"tra_vehicle-lifetime_new_freight_HDVM_PHEV-diesel",
"tra_vehicle-lifetime_new_freight_HDVM_PHEV-gasoline",
"tra_vehicle-lifetime_new_freight_HDVM_ICE-gasoline",
"tra_vehicle-lifetime_new_freight_HDVM_ICE-diesel",
"tra_vehicle-lifetime_new_freight_HDVM_ICE-gas",
"tra_vehicle-lifetime_new_freight_HDVM_BEV",
"tra_vehicle-lifetime_new_freight_HDVH_FCEV",
"tra_vehicle-lifetime_new_freight_HDVH_CEV",
"tra_vehicle-lifetime_new_freight_HDVH_PHEV-diesel",
"tra_vehicle-lifetime_new_freight_HDVH_PHEV-gasoline",
"tra_vehicle-lifetime_new_freight_HDVH_ICE-gasoline",
"tra_vehicle-lifetime_new_freight_HDVH_ICE-diesel",
"tra_vehicle-lifetime_new_freight_HDVH_ICE-gas",
"tra_vehicle-lifetime_new_freight_HDVH_BEV",
"tra_vehicle-lifetime_new_freight_rail_CEV",
"tra_vehicle-lifetime_new_freight_rail_ICE-diesel",
"tra_vehicle-lifetime_new_freight_aviation_BEV",
"tra_vehicle-lifetime_new_freight_aviation_ICE",
"tra_vehicle-lifetime_new_freight_IWW_FCEV",
"tra_vehicle-lifetime_new_freight_IWW_BEV",
"tra_vehicle-lifetime_new_freight_IWW_ICE",
"tra_vehicle-lifetime_new_freight_marine_FCEV",
"tra_vehicle-lifetime_new_freight_marine_BEV",
"tra_vehicle-lifetime_new_freight_marine_ICE",
"tra_vehicle-lifetime_new_passenger_LDV_ICE-diesel",
"tra_vehicle-lifetime_new_passenger_LDV_ICE-gasoline",
"tra_vehicle-lifetime_new_passenger_LDV_ICE-gas",
"tra_vehicle-lifetime_new_passenger_LDV_PHEV-diesel",
"tra_vehicle-lifetime_new_passenger_LDV_PHEV-gasoline",
"tra_vehicle-lifetime_new_passenger_LDV_BEV",
"tra_vehicle-lifetime_new_passenger_LDV_FCEV",
"tra_vehicle-lifetime_new_passenger_2W_ICE-gas",
"tra_vehicle-lifetime_new_passenger_2W_ICE-diesel",
"tra_vehicle-lifetime_new_passenger_2W_ICE-gasoline",
"tra_vehicle-lifetime_new_passenger_2W_BEV",
"tra_vehicle-lifetime_new_passenger_2W_FCEV",
"tra_vehicle-lifetime_new_passenger_2W_PHEV",
"tra_vehicle-lifetime_new_passenger_bus_ICE-diesel",
"tra_vehicle-lifetime_new_passenger_bus_ICE-gasoline",
"tra_vehicle-lifetime_new_passenger_bus_ICE-gas",
"tra_vehicle-lifetime_new_passenger_bus_BEV",
"tra_vehicle-lifetime_new_passenger_bus_FCEV",
"tra_vehicle-lifetime_new_passenger_bus_PHEV-diesel",
"tra_vehicle-lifetime_new_passenger_metro-tram_CEV",
"tra_vehicle-lifetime_new_passenger_rail_CEV",
"tra_vehicle-lifetime_new_passenger_rail_FCEV",
"tra_vehicle-lifetime_new_passenger_rail_ICE-diesel",
"tra_vehicle-lifetime_new_passenger_aviation_ICE",
"tra_vehicle-lifetime_new_passenger_aviation_BEV",
]

In [ ]:
var_df = pd.read_excel(VAR_DF_PATH, sheet_name="input_vars")

In [ ]:
var_df = var_df[~var_df["var_name"].isin(clm_related_vars)]
var_df = var_df[~var_df["var_name"].isin(eucalc_transpor_related_data)]

In [ ]:
# var_df = var_df[var_df["var_name"].isin(["industrial or commercial units cover"])]

In [ ]:
regions_df = gpd.read_file(os.path.join(SHP_PATH, "LAU.shp"))
regions_df["country"] = regions_df["prnt_code"].str[:2]

In [ ]:
nl = '\n'
latex_nl = '\\\\'
esc_nl = '\\'

In [ ]:
replacements = {'begin{tabular}': 'begin{tabularx}{\\textwidth}',
                'end{tabular}': 'end{tabularx}',
                 "& 0 \\\\": "",
                "_": "\_",
                '\\\\': '\\\\\midrule',
                "{\\textwidth}{| X | X |}\n": "{\\textwidth}{| X | X |}\n\midrule"

               }

def get_latex_table(df):
    # prepare column_format
    col_format = "| X |"
    for i in range(len(df.columns)):
        col_format = f"{col_format} X |"
    
    # pd to latex
    table = df.style.to_latex(column_format=col_format, 
                                             position_float="centering",
                                             hrules=False,
                                             environment = "table*", 
                                             position="h"
                                             )
    # replace tabular with tabularx and add width=textwidth
    for key, value in replacements.items():
        table = table.replace(key, value)
        
    return table 

In [ ]:
for country_code in ["DE", "PL", "ES"]:

    for var_name in var_df["var_name"]:
        
        data_df = pd.read_csv(os.path.join(DATA_PATH, f"{country_code}_{var_name}.csv"))
        
        if len(data_df) != 0: # vars present in only one country have a df with 0 rows . need to do something about this in the API
            save_var_name = f"{var_name[0].upper()}{var_name[1:]}"
              
            var_metadata = client.get_variable_metadata(api_key=api_key,
                                                    variable_name=var_name,
                                                    country_code=country_code,
                                                       spatial_resolution="LAU")
            
        
            var_unit = var_metadata["var_unit"]
            org_res = data_df["original_resolution"].unique().item()
            var_description = var_metadata["var_description"]
            data_years = list(data_df["year"].unique())
            data_source = data_df["data_source_name"].unique().item()
        
            # table 
            table_dict = {"Unit of measure": var_unit, 
                    "Original resolution": org_res,
                          }

            if isinstance(data_years[0], np.int64):
                data_years_neat = ", ".join([str(i) for i in data_years])
                table_dict.update({'Data years': data_years_neat})
            else:
                table_dict.update({'Data years': "No year"})
                
            table_dict.update({'Data source': data_source}) #TODO: multirow with data source name, link and citation??  

            if var_description is not None:
                table_dict.update({"Variable description": var_description})

            if org_res == "LAU":
                table_dict.update({"Further disaggregated?": "No"})
            
            else:
                proxies = []
                for proxy_dict in var_metadata["proxy_metrics"]:
                    proxies.append(proxy_dict["var_name"])
                    
                if proxies == []:
                    table_dict.update({"Further disaggregated?": "Yes"})
                    table_dict.update({"Disaggregation method": "Same value as parent region to its LAU regions"})
                
                else:
                    proxy_neat = ", ".join(proxies)
                    
                    table_dict.update({"Further disaggregated?": "Yes"})
                    table_dict.update({"Disaggregation method": "Using proxy variables"})
                    table_dict.update({"Proxy variables": proxy_neat})
                
            table = pd.DataFrame.from_dict(table_dict, orient="index")
            with open(os.path.join(REPORT_PATH,  "tables", f"{save_var_name}_{country_code}.tex"), "w") as f:

                latex_table = get_latex_table(table)
                f.write(latex_table)
                f.write(nl)

            # figure 1: distribution plot =============================================================================

            fig = plt.figure(figsize=(13, 5))
            gs = fig.add_gridspec(1, 2, wspace=0.3, hspace=0)

            # distribution - boxplot -------
            ax1 = plt.subplot(gs[:, :1])

            _data = data_df.loc[data_df["quality_rating"] == "good", "value"].values # only showing "good" values
            sns.boxplot(y= _data,
                        width=0.3, ax=ax1)

            ax1.set_xlabel("")
            ax1.set_ylabel(f"[{var_unit}]", fontsize=17)
            ax1.tick_params(labelsize=15)

            ax1.tick_params(
                axis='x',          # changes apply to the x-axis
                which='both',      # both major and minor ticks are affected
                bottom=False,      # ticks along the bottom edge are off
                top=False,         # ticks along the top edge are off
                labelbottom=False) # labels along the bottom edge are off

            # missing values - barplot --------
            ax2 = plt.subplot(gs[:, 1:])

            n_good = len(data_df[data_df["quality_rating"] == "good"])
            n_bad = len(data_df[data_df["quality_rating"] == "bad"])

            ax2.bar(x=[0, 1, 2], height=[n_good, 0, 0], width=0.5, color='green', label='present')
            ax2.bar(x=[0, 1, 2], height=[n_bad, 0, 0], bottom=n_good, width=0.5, color='red', label='missing')

            ax2.tick_params(labelsize=13)
            ax2.tick_params(
                axis='x',          # changes apply to the x-axis
                which='both',      # both major and minor ticks are affected
                bottom=False,      # ticks along the bottom edge are off
                top=False,         # ticks along the top edge are off
                labelbottom=False) # labels along the bottom edge are off

            ax2.set_ylabel('[number of regions]', fontsize=17)
            ax2.set_ylim([0, len(data_df)+500])

            ax2.tick_params(labelsize=15)

            ax2.set_axisbelow(True)
            ax2.yaxis.grid(color='gray', linestyle='-', alpha=0.3)

            ax2.legend(borderpad=1, loc="upper right", fontsize=15)

            plt.savefig(os.path.join(REPORT_PATH, 
                                     "figures", 
                                     f"{var_name[0].upper()}{var_name[1:]}_{country_code}.png"),
                       format='png', bbox_inches="tight", dpi=200)

            # figure 2: maps =============================================================================
            # sometimes there is a mismatch between types. donno why!
            country_regions_df = regions_df[regions_df["country"] == country_code]

            fixed_code_len = len(country_regions_df["code"].values[0])
            data_df['region_code'] = data_df['region_code'].astype(str) 
            data_df['region_code'] = data_df['region_code'].str.zfill(fixed_code_len)

            _sub_df = pd.merge(country_regions_df, 
                               data_df, 
                               left_on="code", 
                               right_on="region_code", 
                               how="left")

            # distribution --------
            fig = plt.figure(figsize=(13, 13))
            gs = fig.add_gridspec(1, 2, wspace=0.1, hspace=0)

            ax1 = plt.subplot(gs[:, :1])

            vmin, vmax = min(_sub_df['value']), max(_sub_df['value'])
            _sub_df.plot(column='value', cmap='Blues', linewidth=0.8, ax=ax1, edgecolor='none')
            ax1.axis('off')

            sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
            sm._A = []
            axins = inset_axes(ax1,
                        width="90%",  
                        height="5%",
                        loc='lower center',
                        borderpad=-2
                       )
            clb = fig.colorbar(sm, cax=axins, orientation="horizontal", shrink=0.8)
            clb.ax.tick_params(labelsize=15)
            clb.ax.set_title(f"[{var_unit}]", fontsize=15)
            # missing values --------
            ax2 = plt.subplot(gs[:, 1:])

            col_dict = {"good": "green", "bad": "red"}
            map_dict = {"good": "present", "bad": "missing"}
            for key, group in _sub_df.groupby("quality_rating"): 
                group.replace({"quality_rating": key})
                group.plot(column='quality_rating', 
                           color=col_dict[key], 
                           edgecolor=col_dict[key],
                           ax=ax2)

            handles = []
            for col, color in col_dict.items():
                patch = mpatches.Patch(color=color, label=map_dict[col])
                handles.append(patch)

            plt.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.5, 0), ncol=2, fontsize=15)

            ax2.axis('off')

            plt.savefig(os.path.join(REPORT_PATH, 
                                     "figures", 
                                     f"{save_var_name}_{country_code}_map.png"),
                       format='png', bbox_inches="tight", dpi=200)

In [ ]:
# the vars are currently not in the API
var_df = var_df[~var_df["var_name"].isin([
                                        "land use - no data cover",
    "waste from other sources - glass"

])]

In [ ]:
#for latex report - segregate between common var and country-specific vars 
common_vars = []
de_extra_vars = []
pl_extra_vars = []
for var_name in var_df["var_name"]:
    save_var_name = f"{var_name[0].upper()}{var_name[1:]}"
    
    de_df = pd.read_csv(os.path.join(DATA_PATH, f"DE_{var_name}.csv"))
    pl_df = pd.read_csv(os.path.join(DATA_PATH, f"PL_{var_name}.csv"))
    es_df = pd.read_csv(os.path.join(DATA_PATH, f"ES_{var_name}.csv"))
    
    if len(es_df) != 0:
        common_vars.append(save_var_name)
    elif len(pl_df) > 0:
        pl_extra_vars.append(save_var_name)    
    elif len(de_df) > 0:
        de_extra_vars.append(save_var_name)   

In [ ]:
with open('common_vars.txt', 'w') as f:
    neat_str = ", ".join(common_vars)
    neat_str = neat_str.replace("_", "\_")
    f.write(neat_str)
    
with open('pl_extra_vars.txt', 'w') as f:
    neat_str = ", ".join(pl_extra_vars)
    neat_str = neat_str.replace("_", "\_")
    f.write(neat_str)
    
with open('de_extra_vars.txt', 'w') as f:
    neat_str = ", ".join(de_extra_vars)
    neat_str = neat_str.replace("_", "\_")
    f.write(neat_str)